In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
%matplotlib inline

### Notebook zawiera następujące wykresy:

#### Widgety (wybór kraju, wybór wide_category):
1. ilość energii wyprodukowanej przez dany kraj vs średnia produkcja energii top 5 krajów w UE
2. ilościowo dana (wybrana) kategoria per dany kraj (nie jestem pewna czy jest potrzebny? może zostawić tylko udziałowe?)
3. udział poszczególnych źródeł energii w danym kraju (green, nuclear, heat) - bar plot
4. udział poszczególnych źródeł energii w danym kraju (green, nuclear, heat) - bar stack plot

In [ ]:
data_frame = pd.read_csv("energy_gross_prod_UE.csv")
data_frame

In [ ]:
#suma energii dla całej EU per rok w rozbiciu na kategorie
cat_prod_sum = data_frame[["wide_category", "year", "quantity","country_or_area"]].groupby(by = ["wide_category", "year","country_or_area"], as_index = False).sum()
cat_prod_sum

In [ ]:
#średnia produkcja energii per rok w UE (ilościowo)

gross_prod_mean = data_frame[["year", "quantity"]].groupby(by = ["year"], as_index = False).mean()
gross_prod_mean

In [ ]:
#całkowita produkcja per rok kraj (ilościowo)

gross_prod = data_frame[["country_or_area", "year", "quantity"]].groupby(by = ["country_or_area", "year"], as_index = False).sum()
gross_prod

In [ ]:
#top 5 pod względem "quantity" w 2014 (czyli ostatnim okresie w próbie)
gross_prod[gross_prod["year"] == 2014].nlargest(5, "quantity")

In [ ]:
#DF dla średniej z top 5 
#warunki
var_1 = gross_prod["country_or_area"] == "Germany"
var_2 = gross_prod["country_or_area"] == "Spain"
var_3 = gross_prod["country_or_area"] == "United Kingdom"
var_4 = gross_prod["country_or_area"] == "France"
var_5 = gross_prod["country_or_area"] == "Italy"
#DF - wszystkie z top 5
top_5_gross = gross_prod[var_1 | var_2 | var_3 | var_4 | var_5] 
#Groupby po roku ze średnią
top_5_grouped_gross = top_5_gross.groupby(by=["year"], as_index = False)
mean_top_5_gross = pd.DataFrame(top_5_grouped_gross["quantity"].mean())

In [ ]:
#widget
country_list = list(cat_prod_sum["country_or_area"].unique())
category_list = list(cat_prod_sum["wide_category"].unique())
choice_widget_country = widgets.Dropdown(options=country_list)
choice_widget_category = widgets.Dropdown(options=category_list)
values = {"Kraj": choice_widget_country.value, "Kategoria": choice_widget_category.value}
def widgets_handler(country_choice, category_choice):
    values["Kraj"] = country_choice
    values["Kategoria"] = category_choice

In [ ]:
#wykres z sumą energii w totalu w danym kraju (wybranym z listy)
def plot_total():
    x = gross_prod[gross_prod["country_or_area"] == values["Kraj"]]
    plt.figure(figsize=(8, 6))
    plt.plot("year", "quantity", data=x, label = "total energy produced by {}".format(values["Kraj"]))
    plt.plot("year", "quantity", data = mean_top_5_gross, label = "total energy produced by top 5 countries (mean)")
    plt.legend()
    plt.title(label = "total energy produced in {} in comparison with mean of top 5 of UE".format(values["Kraj"]))
    plt.show()

In [ ]:
#wykres z sumą energii w danym kraju (wybranym z listy) i w wybranej kategorii (też z listy)
def plot_category():
    kraj = cat_prod_sum["country_or_area"] == values["Kraj"]
    kategoria = cat_prod_sum["wide_category"] == values["Kategoria"]
    y = cat_prod_sum[kraj & kategoria]
    plt.figure(figsize=(8, 6))
    plt.plot("year", "quantity", data=y)
    plt.title(label = "{} in {}".format(values["Kategoria"], values["Kraj"]))
    plt.show()

In [ ]:
#wykres z totalem produkcji energii w danym kraju oraz udziałem poszczególnych
def plot_bar():
    kraj = cat_prod_sum["country_or_area"] == values["Kraj"]
    kategoria = cat_prod_sum["wide_category"] == values["Kategoria"]
    z = cat_prod_sum[kraj]
    z = z[["year", "wide_category", "quantity"]]
    z_pivot = z.pivot(index = "year", columns = "wide_category", values = "quantity")
    z_pivot = z_pivot.fillna(0)
    z_pivot.plot.bar(stacked = True)
    plt.title(label = "share of energy sources in {}".format(values["Kraj"]))
    plt.legend(title=None)
    plt.show()

In [ ]:
#wykres z totalem produkcji energii w danym kraju oraz udziałem poszczególnych - stack 100%
def plot_bar_100():
    kraj = cat_prod_sum["country_or_area"] == values["Kraj"]
    kategoria = cat_prod_sum["wide_category"] == values["Kategoria"]
    z = cat_prod_sum[kraj]
    z = z[["year", "wide_category", "quantity"]]
    z_pivot = z.pivot(index = "year", columns = "wide_category", values = "quantity")
    z_pivot = z_pivot.fillna(0)
    z_pivot.apply(lambda x: x*100/sum(x), axis=1).plot(kind="bar", stacked=True)
    plt.title(label = "share of energy sources in {}".format(values["Kraj"]))
    plt.legend(title=None)
    plt.show()

In [ ]:
widgets.interact(widgets_handler, country_choice=choice_widget_country, category_choice = choice_widget_category)
plot_total()
plot_category()
plot_bar() #opcja 1
plot_bar_100() #opcja 2